# Projeto 2 - Ciência dos Dados

Nome: Henrique Mualem Marti

Nome: Victor Vergara

Nome: Edgard Neto

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [36]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass.json') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [5]:
#Produto escolhido:
produto = 'coca'

#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 250

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [14]:
file = pd.read_excel('coca.xlsx' , sheet_name='Treinamento')
file.head()

,Treinamento,Relevância
0,olha a hora que é e eu tive que comprar uma co...,1
1,hoje o almoço é c a ingrid....e de quebra uma ...,1
2,acho que vi 7 ggatinhos eita pega amo mais que...,2
3,7h da manhã e eu estava comendo carne assada c...,3
4,mano eu e a layla hoje tomamos uns 6 garrafas ...,2


#### Legenda de classificação:
0: Muito relevante - os *twetts* que expressam opiniões claras e explícitas sobre a bebida "Coca-Cola".

1: Relevante - os *twetts* que expressam alguma opinião sobre a bebida "Coca-Cola".

2: Neutro - os *twetts* que não expressam nenhuma opinião sobre a bebida "Coca-Cola".

3: Inrrelevante - os *twetts* que não estão relacionados a bebida "Coca-Cola".


In [50]:
#Limpeza
texts=file.loc[:, "Treinamento"]
lista_twetts=[]
def cleanup(text):
    
    punctuation = '[!\-.:?;/()_,""''**\n]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

for items in texts:
    p=cleanup(items)
    lista_twetts.append(p)
    
print(lista_twetts)


['olha a hora que é e eu tive que comprar uma coca pq tô com muita vontade de tomar', 'hoje o almoço é c a ingrid    e de quebra uma coca pq ela é viciada nisso jesus kk', 'acho que vi 7 ggatinhos eita pega amo mais que coca  cola amo mais que chocolate @bts twt @claudinorosana @linasnascimento https   t co sprfxbyajo', '7h da manhã e eu estava comendo carne assada com coca cola', 'mano eu e a layla hoje tomamos uns 6 garrafas de dois litros de coca surre', 'só quero uma coca bem gelada', 'sequei a coca que tinha na geladeira  ressaca fdd', '@claytonnerdside perto disso aqui  coca cola é fichinha  p  https   t co nujmsxocqv', 'guaraná ou coca  — coca zero https   t co edghbfl8kn', 'é normal bebê coca uma hora dessa 🤦🏻\u200d♀️', 'rt @ribeirosilva12  porque só falam do bolsonaro e do brasil sobre as queimadas  e não falam do decreto que evo morales assinou para que os…', 'ontem na vulcano pedi um copão de coca  minha mãe me julgou dizendo que não aguento nem minha ressaca   ', '@nemtjf3 

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis (*feito*)
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**